# LogiCAM

In [ ]:
import json, base64
from pathlib import Path
import mimetypes
from PIL import Image, ImageFile
import io
import sys

def load_manifest(path: str):
    data = json.load(open(path, "r"))
    return data if isinstance(data, list) else [data]


ImageFile.LOAD_TRUNCATED_IMAGES = False

def encode_image_base64(image_file: str) -> tuple[str, str]:
    img_path = Path(image_file)
    print(f"[DEBUG] Starting encode for: {img_path}", file=sys.stderr)

    try:
        raw_bytes = img_path.read_bytes()
    except Exception as e:
        print(f"[ERROR] Could not read bytes from {img_path!r}: {e}", file=sys.stderr)
        raise

    mime_type, _ = mimetypes.guess_type(img_path.name)
    if mime_type is None:
        raise ValueError(f"[ERROR] Could not guess MIME type for {img_path!r}")

    if mime_type.lower() in ("image/jpeg", "image/jpg"):
        print(f"[DEBUG] {img_path} is already JPEG", file=sys.stderr)
        b64 = base64.b64encode(raw_bytes).decode("utf-8")
        return b64, "image/jpeg"

    print(f"[DEBUG] Converting {img_path} ({mime_type}) → JPEG…", file=sys.stderr)
    try:
        img = Image.open(io.BytesIO(raw_bytes))
    except Exception as e:
        print(f"[ERROR] PIL.Image.open failed on {img_path!r}: {e}", file=sys.stderr)
        raise

    try:
        rgb = img.convert("RGB")
    except Exception as e:
        print(f"[ERROR] convert('RGB') failed on {img_path!r}: {e}", file=sys.stderr)
        raise

    buffer = io.BytesIO()
    try:
        rgb.save(buffer, format="JPEG", quality=85, optimize=True)
    except Exception as e:
        print(f"[ERROR] rgb.save() failed for {img_path!r}: {e}", file=sys.stderr)
        raise

    jpeg_bytes = buffer.getvalue()
    b64 = base64.b64encode(jpeg_bytes).decode("utf-8")
    print(f"[DEBUG] Conversion successful for: {img_path}", file=sys.stderr)
    return b64, "image/jpeg"

def make_prompt(item: dict) -> str:
    ctx = item["full_context"].strip()
    q = item["question"].strip()

    if item.get("type") == "multiple_choice":
        choices = "\n".join(f"- {c}" for c in item["choices"])
        prompt_body = f"{ctx}\n\n{q}\n\nChoices:\n{choices}"
    else:
        prompt_body = f"{ctx}\n\n{q}"
    
    with open("./prompts/logicam/prompts.txt", "r") as f:
        prompt_template = f.read()
    
    prompt_body = prompt_template + prompt_body
    
    prompt_body += (
        "\n\nPlease provide the answer in the following format:\n"
        "Reasoning: <your reasoning>"
        "Answer: <your answer>\n"
        "Please put your answer in a curly bracket, e.g., {True}.\n"
    )

    return prompt_body

manifest = load_manifest("./data/muslr.json")
print("Number of instances loaded: ", len(manifest))

In [ ]:
import os, json
from pathlib import Path

OPENAI_MODELS = ["gpt-4.1-2025-04-14"]

for model in OPENAI_MODELS:

    out_path = f"./logicam_requests_{model}.jsonl"
    counter = 0
    with open(out_path, "w") as fout:
        for item in manifest:
            img_uri, _ = encode_image_base64(item["image_file"])
            prompt      = make_prompt(item)
            full_prompt = f"{img_uri}\n\n{prompt}"

            request = {
                "custom_id": item["id"],
                "method":    "POST",
                "url":       "/v1/chat/completions",
                "body": {
                    "model":       model,
                    "messages":   [{"role": "user", "content": full_prompt}],
                    "max_tokens": 4096,
                    "temperature": 0.0,
                }
            }
            fout.write(json.dumps(request) + "\n")
            counter += 1

    print(f"Wrote {counter} requests to {out_path}")
    


data = []
with open('./logicam/logicam_request.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        data.append(json.loads(line))

print(f"Number of requests loaded: {len(data)}")
print(data[0])

In [ ]:

import openai
import json

openai.api_key = '<OPENAI_API_KEY>'
client = openai.Client(api_key=openai.api_key)

def submit_batch(batch_path, desc=""):
    path = batch_path
    batch_input_file = client.files.create(
    file=open(path, "rb"),
    purpose="batch"
    )

    batch_input_file_id = batch_input_file.id

    print("Batch id: ", batch_input_file_id)

    batch_request = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
        "description": desc
        }
    )

    print("Batch request: ", batch_request)

def cancel_batch(batch_id):
    client.batches.cancel(batch_id)
    print("Batch cancelled: ", batch_id)

def check_batch(batch_id):
    batch = client.batches.retrieve(batch_id)
    print("Batch: ", batch)

def retrieve_batch(file_id, save_path):
    file_response = client.files.content(file_id)
    response_text = file_response.text

    responses = [json.loads(line) for line in response_text.strip().split('\n')]

    output_path = save_path
    with open(output_path, 'w') as outfile:
        json.dump(responses, outfile, indent=2)

    print(f"Responses saved to {output_path}")

In [ ]:
# # submit batch
# logicam_batch = submit_batch(out_path, desc="test")

# # check batch status
# batch_id = '<BATCH_ID>'
# batch_info = check_batch(batch_id)

# # retrieve batch results
# output_file_id = batch_info.get("output_file_id")
# retrieve_batch(output_file_id, './results/logicam_results.json')